In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path
import os

In [2]:
path_of_saved_model=Path("./trained_model.h5")
model=tf.keras.models.load_model(path_of_saved_model)

In [3]:
path_of_saved_model

WindowsPath('trained_model.h5')

In [4]:
converter=tf.lite.TFLiteConverter.from_keras_model(model)

In [5]:
tflite_model=converter.convert()

INFO:tensorflow:Assets written to: C:\Users\kulki\AppData\Local\Temp\tmpy_93ct1i\assets


INFO:tensorflow:Assets written to: C:\Users\kulki\AppData\Local\Temp\tmpy_93ct1i\assets


In [7]:
tflight_model_dir=Path("./tflight_model_dir")
tflight_model_dir.mkdir(exist_ok=True, parents=True)

In [8]:
tflight_model_dir

WindowsPath('tflight_model_dir')

In [9]:
tflight_model_dir/"vgg16_model.tflight"

WindowsPath('tflight_model_dir/vgg16_model.tflight')

In [11]:
tflight_model_file=tflight_model_dir/"vgg16_model.tflight"
tflight_model_file.write_bytes(tflite_model)

59068200

In [12]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

tflite_model_quant_file = tflite_model_dir/"vgg16_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)

INFO:tensorflow:Assets written to: C:\Users\kulki\AppData\Local\Temp\tmpva2wtcpm\assets


INFO:tensorflow:Assets written to: C:\Users\kulki\AppData\Local\Temp\tmpva2wtcpm\assets


14837328

In [13]:
14837328/59068200

0.2511897772405457

In [14]:
interpreter=tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter.allocate_tensors()

In [34]:
interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter_quant.allocate_tensors()

In [15]:
import matplotlib.pyplot as plt
test_img=plt.imread("./data/unzip_data_dir/hymenoptera_data/val/ants/800px-Meat_eater_ant_qeen_excavating_hole.jpg")

In [16]:
test_img.shape

(534, 800, 3)

In [17]:
interpreter.get_input_details()

[{'name': 'serving_default_input_2:0',
  'index': 0,
  'shape': array([  1, 224, 224,   3]),
  'shape_signature': array([ -1, 224, 224,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [18]:
interpreter.get_input_details()[0]

{'name': 'serving_default_input_2:0',
 'index': 0,
 'shape': array([  1, 224, 224,   3]),
 'shape_signature': array([ -1, 224, 224,   3]),
 'dtype': numpy.float32,
 'quantization': (0.0, 0),
 'quantization_parameters': {'scales': array([], dtype=float32),
  'zero_points': array([], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [19]:
interpreter.get_input_details()[0]["shape"]

array([  1, 224, 224,   3])

In [20]:
interpreter.get_input_details()[0]["shape_signature"]

array([ -1, 224, 224,   3])

In [21]:
resized_img=tf.image.resize(test_img,(224,224))

In [24]:
tf.image.resize(test_img,(224,224)).shape

TensorShape([224, 224, 3])

In [25]:
input_data=tf.expand_dims(resized_img, axis=0)

In [27]:
input_index=interpreter.get_input_details()[0]["index"]
output_index=interpreter.get_output_details()[0]["index"]

In [26]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 50,
  'shape': array([1, 2]),
  'shape_signature': array([-1,  2]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [29]:
interpreter.set_tensor(input_index, input_data)
interpreter.invoke()
pred=interpreter.get_tensor(output_index)

In [30]:
label_map={0:'ants', 1:'bees'}

In [31]:
argmax=tf.argmax(pred[0]).numpy()

In [32]:
label_map[argmax]

'ants'

In [35]:
input_index = interpreter_quant.get_input_details()[0]["index"]
output_index = interpreter_quant.get_output_details()[0]["index"]

interpreter_quant.set_tensor(input_index, input_data)
interpreter_quant.invoke()
pred = interpreter_quant.get_tensor(output_index)

argmax = tf.argmax(pred[0]).numpy()
label_map[argmax]

'ants'